# RewardFunction6
```
Key Elements:
1.Identical to RewardFunction_7 except for curvature calculation
    - uses triangle (waypoint) calculations (where curvature value > .97 means that the track is straight)
2. Other reward values are adjusted (for instance, more emphasis in placed on being closer to the center of the track 
   and rewards for speed have changed)
```

In [2]:
def reward_function(params):
    '''
    Curvature values are based on "triangle" (waypoint calculations)
    '''
   
    # Read input parameters
    #x_car = params['x']
    #y_car = params['y']
    on_track = params['all_wheels_on_track']
    distance_from_center = params['distance_from_center']
    track_width = params['track_width']
    progress = params['progress']/100
    waypoints = params['waypoints']
    closest_waypoints = params['closest_waypoints']
    steering = abs(params['steering_angle']) # Only need the absolute steering angle
    speed = params['speed']
    steps = params["steps"]
    
    REWARD_MIN = 0.1
    REWARD_MAX = 100
    TARGET_STEPS = 150
    
    # Calculate 3 marks that are farther and father away from the center line
    marker_1 = 0.25 * track_width
    marker_2 = 0.4 * track_width
    marker_3 = 0.6 * track_width
    
    # Set Base Reward
    if not on_track: # Fail them if off Track
        return REWARD_MIN
    elif progress == 1:
        #the lap is complete.  if we use more steps than TARGET_STEPS, the reward is lower
        return REWARD_MAX * TARGET_STEPS / steps
    elif steps < 3:
        reward = REWARD_MAX
    else:        
        reward = REWARD_MAX * max(progress, 0.4)
    
    # Determine waypoints
    prev_pt = waypoints[closest_waypoints[0]] # closest waypoint behind car
    next_pt = waypoints[closest_waypoints[1]] # closest waypoint ahead of car
    nextnext_pt = waypoints[(closest_waypoints[1] + 1) % len(waypoints)] # waypoint after next_pt
    
    prev_next_dist = ((prev_pt[0] - next_pt[0])**2 + (prev_pt[1] - next_pt[1])**2)**0.5    
    next_nextnext_dist = ((next_pt[0] - nextnext_pt[0])**2 + (next_pt[1] - nextnext_pt[1])**2)**0.5   
    prev_nextnext_dist = ((prev_pt[0] - nextnext_pt[0])**2 + (prev_pt[1] - nextnext_pt[1])**2)**0.5
    #car_next_dist = ((x_car - next_pt[0])**2 + (y_car - next_pt[1])**2)**0.5    
    #car_nextnext_dist = ((x_car - nextnext_pt[0])**2 + (y_car - nextnext_pt[1])**2)**0.5
    
    # Find out whether track is straight or curved
    if prev_nextnext_dist/(prev_next_dist + next_nextnext_dist) > .97:
        # track is straight
        if speed > 4:
            reward -= 1
        elif speed < 3:
            reward -= 1
        else:
            reward += 2
        
        # Give higher reward if the car is closer to center line 
        if distance_from_center <= marker_1:
            reward += 5
        elif distance_from_center <= marker_2:
            reward += 3
        elif distance_from_center <= marker_3:
            reward += 1
            
        if steering < 15:
            reward += 1
    
    else:
        #track is curved
        if distance_from_center <= marker_3:
            reward += 5
        if speed > 2:
            reward += 3
        if steering < 30:
            reward += 5
    
    return float(reward)
